## first extract transcript


In [241]:
%load_ext autotime

time: 316 µs (started: 2024-05-25 11:48:57 +05:45)


In [1]:

import pickle
\
# Load the object from the file
with open('my_variable.pkl', 'rb') as file:
    transcript = pickle.load(file)

In [2]:
new_t = []
for item in transcript.words:
    new_t.append({ 'word': item['word'], 'round' : round(item['start'], 2), 'start' : item['start'], 'end': item['end']})

In [3]:
new_t

[{'word': 'Boom',
  'round': 1.8,
  'start': 1.7999999523162842,
  'end': 2.2799999713897705},
 {'word': 'All',
  'round': 2.28,
  'start': 2.2799999713897705,
  'end': 2.640000104904175},
 {'word': 'right',
  'round': 2.64,
  'start': 2.640000104904175,
  'end': 2.7200000286102295},
 {'word': "We're",
  'round': 2.8,
  'start': 2.799999952316284,
  'end': 2.9800000190734863},
 {'word': 'live',
  'round': 2.98,
  'start': 2.9800000190734863,
  'end': 3.259999990463257},
 {'word': 'Thank',
  'round': 3.48,
  'start': 3.4800000190734863,
  'end': 3.859999895095825},
 {'word': 'you',
  'round': 3.86,
  'start': 3.859999895095825,
  'end': 3.940000057220459},
 {'word': 'very',
  'round': 3.94,
  'start': 3.940000057220459,
  'end': 4.119999885559082},
 {'word': 'much',
  'round': 4.12,
  'start': 4.119999885559082,
  'end': 4.179999828338623},
 {'word': 'for',
  'round': 4.18,
  'start': 4.179999828338623,
  'end': 4.440000057220459},
 {'word': 'doing',
  'round': 4.44,
  'start': 4.440000

## divide transcript into 90 sec intervals


In [4]:


# Initialize variables
two_minutes = 90  # 2 minutes in seconds
chunks = []
current_chunk = []
start_time = new_t[0]['start']  # Start time of the first word

# Iterate through the list
for item in new_t:
    word, timestamp = item['word'], item['round']
    
    if timestamp - start_time < two_minutes:
        current_chunk.append((word, timestamp))
    else:
        chunks.append(current_chunk)
        current_chunk = [(word, timestamp)]
        start_time = timestamp

# Add the last chunk if it has words
if current_chunk:
    chunks.append(current_chunk)

# Unpack chunks into individual variables
chunk1, chunk2, chunk3 = chunks[:3]  # Adjust the number of chunks based on your need




# Note: This will print only the first 3 chunks. Adjust the code if you need more or fewer chunks.


## extract the catchy parts

In [214]:
import asyncio
from openai import AsyncOpenAI
import time
import ast

client = AsyncOpenAI()

async def extract(prev, curr, nextt):
    
    
    output_format = """{
    "topic" : answer
    "start": start_time_in_seconds,
    "end" : end_time_in_seconds
    }"""
    
    input_prompt = f"""

    
    You are given an audio transcript in this format [('hello', 0), ('my', 1), ('name', 2)]

    the words are in consecutive order. The first index of each tuple contains the actual word being spoken and
    second index inside each tuple contains the timestamp at which the word is being spoken.
    
    Your task is to identify interesting, catchy topic being discussed in the transcript and give me the start and end timestamp
    for that topic and the topic name. Remember the duration of the topic being discussed should be between 60-90 seconds.
    
    The start timestamps should be catchy and should build up the initial words for our main topic that comes afterwards.
    The end timestamp should stop as soon as the topic ends, or when it starts to become verbose.
    
    You'll be given 3 values.
    Previous (list) : The transcript of previous timeframe.
    Current (list): The transcript of current timeframe.
    Next (list) : The transcript of next timeframe.
    
    Mainly, you should focus on "Current" transcript but it may contain truncated informations for which you can use
    the "Previous" and "Next" transcript.
    
    If the topic being discussed is incomplete then please add none to all the keys in the final output JSON
    
    Your output shouldf be in this JSON format
    
    {output_format}
    


    ### Input:
    
    Previous : {prev}
    Current : {curr}
    Next : {nextt}
    

    ### OUTPUT: 

    """

    messages = [
        {"role": "system", "content": """ You are an Intelligent agent that identifies interesting topic being discussed 
        in the text."""},
        {"role": "user", "content": input_prompt},
    ]
    
    chat_completion =  await client.chat.completions.create(messages=messages, model='gpt-3.5-turbo-0125', response_format={ "type": "json_object" })
    return chat_completion.choices[0].message.content



In [6]:

async def generate_batch(chunks):

    tasks = []
    end = (int(len(chunks)/3)*3)
    for ind in range(0,end, 3):
        print(ind)

        prev = chunks[ind]
        curr = chunks[ind+1]
        nextt = chunks[ind+2]

        tasks.append(extract(prev,curr,nextt))


    results = await asyncio.gather(*tasks)
    return results



In [7]:
result = await generate_batch(chunks)

0
3
6
9


In [8]:
new_result = []
for item in result:
    stripped_item = item.strip('```')
    stripped_item = stripped_item.strip('json')
    new_result.append(ast.literal_eval(stripped_item))

In [201]:
new_result

[{'topic': 'Life and Learning Journey',
  'start': 92.42,
  'end': 182.08,
  'transcript': [{'word': 'you',
    'round': 92.42,
    'start': 92.41999816894531,
    'end': 92.41999816894531},
   {'word': 'know',
    'round': 92.42,
    'start': 92.41999816894531,
    'end': 92.73999786376953},
   {'word': 'thinker',
    'round': 92.74,
    'start': 92.73999786376953,
    'end': 93.5199966430664},
   {'word': 'et',
    'round': 93.64,
    'start': 93.63999938964844,
    'end': 93.73999786376953},
   {'word': 'cetera',
    'round': 93.74,
    'start': 93.73999786376953,
    'end': 93.83999633789062},
   {'word': 'So',
    'round': 94.58,
    'start': 94.58000183105469,
    'end': 94.91999816894531},
   {'word': 'I',
    'round': 94.92,
    'start': 94.91999816894531,
    'end': 95.23999786376953},
   {'word': 'like',
    'round': 95.24,
    'start': 95.23999786376953,
    'end': 95.54000091552734},
   {'word': 'the',
    'round': 95.54,
    'start': 95.54000091552734,
    'end': 96.059997

In [10]:
for index in range(len(new_result)):
    new_result[index]['transcript'] = []
    

for index, each_result in enumerate(new_result):
    for ind, item in enumerate(new_t):
        if item['round'] >= each_result['start'] and item['round'] <= each_result['end']:
            new_result[index]['transcript'].append(item)

new_result            
  

[{'topic': 'Life and Learning Journey',
  'start': 92.42,
  'end': 182.08,
  'transcript': [{'word': 'you',
    'round': 92.42,
    'start': 92.41999816894531,
    'end': 92.41999816894531},
   {'word': 'know',
    'round': 92.42,
    'start': 92.41999816894531,
    'end': 92.73999786376953},
   {'word': 'thinker',
    'round': 92.74,
    'start': 92.73999786376953,
    'end': 93.5199966430664},
   {'word': 'et',
    'round': 93.64,
    'start': 93.63999938964844,
    'end': 93.73999786376953},
   {'word': 'cetera',
    'round': 93.74,
    'start': 93.73999786376953,
    'end': 93.83999633789062},
   {'word': 'So',
    'round': 94.58,
    'start': 94.58000183105469,
    'end': 94.91999816894531},
   {'word': 'I',
    'round': 94.92,
    'start': 94.91999816894531,
    'end': 95.23999786376953},
   {'word': 'like',
    'round': 95.24,
    'start': 95.23999786376953,
    'end': 95.54000091552734},
   {'word': 'the',
    'round': 95.54,
    'start': 95.54000091552734,
    'end': 96.059997

## combine it for every 5 seconds

In [12]:
# Function to combine words in 5-second intervals
def combine_words(transcript, interval=5):
    combined_transcript = []
    transcript.sort(key=lambda x: x['start'])
    
    start_time = transcript[0]['start']
    end_time = start_time + interval
    
    current_chunk = {'start': start_time, 'end': end_time, 'words': []}
    
    for item in transcript:
        if item['start'] >= end_time:
            combined_transcript.append(current_chunk)
            start_time = item['start']
            end_time = start_time + interval
            current_chunk = {'start': start_time, 'end': end_time, 'words': []}
        
        current_chunk['words'].append(item['word'])
    
    combined_transcript.append(current_chunk)  # Append the last chunk
    
    return combined_transcript



In [204]:
interval_words = (combine_words(new_result[0]['transcript']))

In [206]:
input_dict = {num: " ".join(item['words']) for num,item in enumerate(combine_words(new_result[0]['transcript']))}

input_dict

{0: 'you know thinker et cetera So I like the model of life that the',
 1: "ancients had the Greeks the Romans right where you would start out and when you're young you're",
 2: "just like going to school Then you're going to war Then you're running a business Then you're supposed",
 3: "to serve in the Senate or the government Then you become a philosopher There's sort of this arc to life where",
 4: 'you try your hand at everything And as one of my friends says specialization',
 5: "is for insects right So everyone should just be able to do everything And so I don't believe",
 6: "in this model anymore of trying to focus your life down on one thing You've got one life",
 7: "Just do everything you're going to do I couldn't agree more And I think that",
 8: 'sometimes people find certain success in whatever the endeavor is and then they think that that is their',
 9: "niche and they stick with it and they never change And they're almost out of fear Well",
 10: "it's hard because there

In [215]:

async def identify_images(input_dict):
    
    
    output_format = """{
    "start": start_time_in_seconds, // edited start time
    "end" : end_time_in_seconds, //edited end time
    }"""
    
    input_prompt = """ You are dictionary like this 
    
    {'1': "Greeks the Romans right where you would start out and" , '2' : "when you're young you're just like going to school Then you're going" }
    your task is to find out which of these texts can be 
    formed into an meaningful keyword.
    
    For example:
    If the text is : "Greeks the Romans right where you would start out and"
    
    The text can have meaningful keyword like: "Ancient Romans"
    
    So your task is to formulate a simple two word keyword for the for each of keys the dictionary and output it in JSON format
    
    like this:
    
    
    {'1': "Ancient Rome" , '2' : "School Boy" }
    
    You must try to find keyword for each of the keys but if the texts doesn't contain any meaningful information to be turned into meaningful keyword then in such case 
    please output None for that key.

    """

    input_variable = f"""
    
    ### Input Dictionary:
    
    {input_dict}
    """

    messages = [
        {"role": "system", "content": """ You are an Intelligent agent responsible for constructing meaningful title from a given text"""},
        {"role": "user", "content": input_prompt +  input_variable},
    ]
    
    chat_completion =  await client.chat.completions.create(messages=messages, model='gpt-3.5-turbo-0125', response_format={ "type": "json_object" })
    return chat_completion.choices[0].message.content


In [227]:
ans = await identify_images(input_dict)

In [228]:
import json

ans = json.loads(ans)

In [229]:
for index, item in enumerate(interval_words):
    print(index,ans.get(str(index), None) )
    interval_words[index]['keywords'] = ans.get(str(index), None)

0 None
1 Ancient Wisdom
2 School Progression
3 Life's Journey
4 Versatile Life
5 None
6 Multiple Passions
7 Embrace Variety
8 Success Trap
9 Fearful Stagnation
10 None
11 Unfinished Ascent
12 None
13 Finding New Paths
14 Avoiding Restart
15 Limited Time Choices
16 Creators' Renewal
17 Artistic Resilience


In [219]:
interval_words

[{'start': 92.41999816894531,
  'end': 97.41999816894531,
  'words': ['you',
   'know',
   'thinker',
   'et',
   'cetera',
   'So',
   'I',
   'like',
   'the',
   'model',
   'of',
   'life',
   'that',
   'the'],
  'keywords': None},
 {'start': 97.5999984741211,
  'end': 102.5999984741211,
  'words': ['ancients',
   'had',
   'the',
   'Greeks',
   'the',
   'Romans',
   'right',
   'where',
   'you',
   'would',
   'start',
   'out',
   'and',
   'when',
   "you're",
   'young',
   "you're"],
  'keywords': 'Ancient Life'},
 {'start': 102.80000305175781,
  'end': 107.80000305175781,
  'words': ['just',
   'like',
   'going',
   'to',
   'school',
   'Then',
   "you're",
   'going',
   'to',
   'war',
   'Then',
   "you're",
   'running',
   'a',
   'business',
   'Then',
   "you're",
   'supposed'],
  'keywords': 'Life Stages'},
 {'start': 107.94000244140625,
  'end': 112.94000244140625,
  'words': ['to',
   'serve',
   'in',
   'the',
   'Senate',
   'or',
   'the',
   'government'

## find index to add images

In [220]:
len(interval_words)

18

In [221]:
int(18*0.2)

3

In [230]:
import random

index_to_find_images = []


for index in range(0+1, len(interval_words)-3, 4):
    print('index',index, index + 1 ,index + 3 - 1 )
    number = random.randint(index + 1, index + 3 - 1)
    print(number)
    
    if interval_words[number]['keywords'] is not None:
        index_to_find_images.append({number: interval_words[number]})


index 1 2 3
2
index 5 6 7
6
index 9 10 11
10
index 13 14 15
14


In [231]:
index_to_find_images

[{2: {'start': 102.80000305175781,
   'end': 107.80000305175781,
   'words': ['just',
    'like',
    'going',
    'to',
    'school',
    'Then',
    "you're",
    'going',
    'to',
    'war',
    'Then',
    "you're",
    'running',
    'a',
    'business',
    'Then',
    "you're",
    'supposed'],
   'keywords': 'School Progression'}},
 {6: {'start': 123.73999786376953,
   'end': 128.73999786376953,
   'words': ['in',
    'this',
    'model',
    'anymore',
    'of',
    'trying',
    'to',
    'focus',
    'your',
    'life',
    'down',
    'on',
    'one',
    'thing',
    "You've",
    'got',
    'one',
    'life'],
   'keywords': 'Multiple Passions'}},
 {14: {'start': 164.72000122070312,
   'end': 169.72000122070312,
   'words': ['wants',
    'to',
    'do',
    'that',
    'People',
    "don't",
    'want',
    'to',
    'start',
    'over',
    'And',
    "it's",
    'the',
    'nature',
    'of',
    'later',
    'in',
    'life',
    'that',
    'you',
    'just',
    "do

## get image

In [232]:
import aiohttp
import asyncio
from PIL import Image
from io import BytesIO

async def fetch_image_id(session, keyword):
    url = f"https://api.pexels.com/v1/search?query={keyword}&per_page=1&orientation=portrait&size=small"
    headers = {
        "Authorization": "DS6m37FqnoQRq8JKFy7J0hnmg1yqQvzImxNHRGjhfLgE8qsQtgDi11U9"
    }

    async with session.get(url, headers=headers) as response:
        if response.status == 200:
            # Parse the response JSON asynchronously
            data = await response.json()
            photo_url = data['photos'][0]['src']['portrait']

            # Make a request to get the photo
            async with session.get(photo_url) as photo_response:
                if photo_response.status == 200:
                    # Read the image content
                    photo_data = await photo_response.read()

                    # Open the image
                    image = Image.open(BytesIO(photo_data))

                    # Save the image
                    image.save(f"images/{keyword}.jpeg")
                    print(f"Image for keyword '{keyword}' saved as '{keyword}.jpeg'")
                else:
                    print(f"Failed to retrieve the photo: {photo_response.status}")
        else:
            print(f"Failed to retrieve data for {keyword}: {response.status}")

async def get_image_ids(keywords):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_image_id(session, keyword) for keyword in keywords]
        await asyncio.gather(*tasks)


In [233]:
keywords = []
for item in index_to_find_images:
    for _,val in item.items():
        keywords.append(val['keywords'])
        
final_images = await get_image_ids(keywords)

Image for keyword 'Multiple Passions' saved as 'Multiple Passions.jpeg'
Image for keyword 'Avoiding Restart' saved as 'Avoiding Restart.jpeg'
Image for keyword 'School Progression' saved as 'School Progression.jpeg'


## resize and crop

In [154]:

from PIL import Image

def resize_and_crop(image_path,desired_height = 720 ):
    
    img = Image.open(image_path)

#     desired_height = 720  # Adjust this to your desired height
    width, height = img.size
    ratio = height / desired_height
    new_width = int(width / ratio)

    resized_img = img.resize((new_width, desired_height))

# #     resized_image_path = "images/resized.jpg"
#     resized_img.save(image_path)

    
    # Open the image
    image = resized_img

    # Get the original width and height
    width, height = resized_img.size

    # Define the new desired width
    new_width = (9/16)*height  # Adjust this value as needed

    # Calculate the left and right crop positions to maintain aspect ratio
    left = int((width - new_width) / 2)
    right = left + new_width

    # Create the crop box using (left, top, right, bottom)
    crop_box = (left, 0, right, height)

    # Crop the image
    cropped_image = image.crop(crop_box)

    # Save the cropped image (optional)
    cropped_image.save(image_path)
    # Optionally display the resized image
    # resized_img.show()


## zoom

In [234]:
def Zoom(clip,mode='in',position='center',speed=1):
    fps = clip.fps
    duration = clip.duration
    total_frames = int(duration*fps)
    def main(getframe,t):
        frame = getframe(t)
        h,w = frame.shape[:2]
        i = t*fps
        if mode == 'out':
            i = total_frames-i
        zoom = 1+(i*((0.1*speed)/total_frames))
        positions = {'center':[(w-(w*zoom))/2,(h-(h*zoom))/2],
                     'left':[0,(h-(h*zoom))/2],
                     'right':[(w-(w*zoom)),(h-(h*zoom))/2],
                     'top':[(w-(w*zoom))/2,0],
                     'topleft':[0,0],
                     'topright':[(w-(w*zoom)),0],
                     'bottom':[(w-(w*zoom))/2,(h-(h*zoom))],
                     'bottomleft':[0,(h-(h*zoom))],
                     'bottomright':[(w-(w*zoom)),(h-(h*zoom))]}
        tx,ty = positions[position]
        M = np.array([[zoom,0,tx], [0,zoom,ty]])
        frame = cv2.warpAffine(frame,M,(w,h))
        return frame
    return clip.fl(main)

## crop and make video

In [45]:

from moviepy.editor import VideoFileClip, ImageClip
import numpy as np

# from moviepy.editor import *
import cv2
# import numpy as np

for item in index_to_find_images:
    key, val = next(iter(item.items()))

    

    # Load the audio from another video file
    audio_clip = VideoFileClip('Video.mp4').subclip(val['start'], val['end']).audio


    img = 'images/' + val['keywords'] + '.jpeg'
    
    resize_and_crop(img)
    clip = ImageClip(img).set_fps(30).set_duration(val['end'] - val['start'])
    clip = Zoom(clip,mode='in',position='center',speed=2) #zoom function above
    # Set the audio to the image clip
    clip = clip.set_audio(audio_clip)
    clip.write_videofile(f'subclips/{key}_{key}.mp4',preset='superfast')
    
    

Moviepy - Building video subclips/4_4.mp4.
MoviePy - Writing audio in 4_4TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/4_4.mp4



Moviepy - Done !
Moviepy - video ready subclips/4_4.mp4
Moviepy - Building video subclips/9_9.mp4.
MoviePy - Writing audio in 9_9TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/9_9.mp4



Moviepy - Done !
Moviepy - video ready subclips/9_9.mp4
Moviepy - Building video subclips/14_14.mp4.
MoviePy - Writing audio in 14_14TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/14_14.mp4



Moviepy - Done !
Moviepy - video ready subclips/14_14.mp4
Moviepy - Building video subclips/17_17.mp4.
MoviePy - Writing audio in 17_17TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/17_17.mp4



Moviepy - Done !
Moviepy - video ready subclips/17_17.mp4


In [142]:
len(interval_words) - 1

17

In [235]:
prev = 0

breaking_points = []
for item in index_to_find_images:
    key, val = next(iter(item.items()))
    
    breaking_points.append((prev,key-1)) #start, end
    breaking_points.append((key,key)) #images
    
    prev = key+1
    
if breaking_points[-1] != (len(interval_words) - 1, len(interval_words) - 1):
    breaking_points.append((prev,len(interval_words) - 1))

In [236]:
breaking_points

[(0, 1), (2, 2), (3, 5), (6, 6), (7, 13), (14, 14), (15, 17)]

In [255]:
for index, item in enumerate(breaking_points):
    first_start = interval_words[item[0]]['start']
    second_end = interval_words[item[1]]['end']
    
    
    if item[0] != item[1]:     
        print(first_start, second_end , 'gg' , breaking_points[index])
    
    
    else:
        
    #         clip = ImageClip(img).set_fps(30).set_duration(val['end'] - val['start'])
        prev_index_end = interval_words[breaking_points[index-1][1]]['end']
        current_end = interval_words[breaking_points[index+1][0]]['start']
        
        print(prev_index_end , current_end )
    

92.41999816894531 102.5999984741211 gg (0, 1)
102.5999984741211 107.94000244140625
107.94000244140625 123.72000122070312 gg (3, 5)
123.72000122070312 128.9199981689453
128.9199981689453 164.66000366210938 gg (7, 13)
164.66000366210938 169.74000549316406
169.74000549316406 185.05999755859375 gg (15, 17)
time: 1.44 ms (started: 2024-05-25 12:11:47 +05:45)


In [253]:
interval_words

[{'start': 92.41999816894531,
  'end': 97.41999816894531,
  'words': ['you',
   'know',
   'thinker',
   'et',
   'cetera',
   'So',
   'I',
   'like',
   'the',
   'model',
   'of',
   'life',
   'that',
   'the'],
  'keywords': None},
 {'start': 97.5999984741211,
  'end': 102.5999984741211,
  'words': ['ancients',
   'had',
   'the',
   'Greeks',
   'the',
   'Romans',
   'right',
   'where',
   'you',
   'would',
   'start',
   'out',
   'and',
   'when',
   "you're",
   'young',
   "you're"],
  'keywords': 'Ancient Wisdom'},
 {'start': 102.80000305175781,
  'end': 107.80000305175781,
  'words': ['just',
   'like',
   'going',
   'to',
   'school',
   'Then',
   "you're",
   'going',
   'to',
   'war',
   'Then',
   "you're",
   'running',
   'a',
   'business',
   'Then',
   "you're",
   'supposed'],
  'keywords': 'School Progression'},
 {'start': 107.94000244140625,
  'end': 112.94000244140625,
  'words': ['to',
   'serve',
   'in',
   'the',
   'Senate',
   'or',
   'the',
   'go

time: 9.09 ms (started: 2024-05-25 12:10:49 +05:45)


## crop whole video to portrait size

In [ ]:
ffmpeg_extract_subclip("main.mp4", 0, 240, targetname="test1.mp4")

import subprocess

# Define the ffmpeg command
command = ["ffmpeg", "-i", "test1.mp4", "-filter:v", "crop=405:720", "-c:v", "libx264", "-crf", "22", "subclips/cropped.mp4"]

# Execute the command using subprocess.run
subprocess.run(command)

print("FFmpeg command executed successfully!")


[{'start': 92.41999816894531,
  'end': 97.41999816894531,
  'words': ['you',
   'know',
   'thinker',
   'et',
   'cetera',
   'So',
   'I',
   'like',
   'the',
   'model',
   'of',
   'life',
   'that',
   'the'],
  'keywords': 'Smart Interaction'},
 {'start': 97.5999984741211,
  'end': 102.5999984741211,
  'words': ['ancients',
   'had',
   'the',
   'Greeks',
   'the',
   'Romans',
   'right',
   'where',
   'you',
   'would',
   'start',
   'out',
   'and',
   'when',
   "you're",
   'young',
   "you're"],
  'keywords': 'Weird Challenge'},
 {'start': 102.80000305175781,
  'end': 107.80000305175781,
  'words': ['just',
   'like',
   'going',
   'to',
   'school',
   'Then',
   "you're",
   'going',
   'to',
   'war',
   'Then',
   "you're",
   'running',
   'a',
   'business',
   'Then',
   "you're",
   'supposed'],
  'keywords': 'Social Boost'},
 {'start': 107.94000244140625,
  'end': 112.94000244140625,
  'words': ['to',
   'serve',
   'in',
   'the',
   'Senate',
   'or',
   'th

In [289]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

for index, item in enumerate(breaking_points):
    first_start = interval_words[item[0]]['start']
    second_end = interval_words[item[1]]['end']
    

    
    video = VideoFileClip("subclips/cropped.mp4")
    
    print(index)
    if item[0] != item[1]:
        print(first_start)
        print(second_end)
        ffmpeg_extract_subclip("subclips/cropped.mp4", first_start, second_end -2, targetname=f"subclips/{item[0]}_{item[1]}.mp4")
        ffmpeg_extract_subclip("subclips/cropped.mp4", second_end -2, second_end, targetname=f"subclips/end_{item[0]}_{item[1]}.mp4")

        # Extract the subclips
        clip1 = video.subclip(first_start, second_end -2)
        clip2 = video.subclip(second_end -2, second_end)

        # Write the subclips to files (optional, for verification)
        clip1.write_videofile(f"subclips/{item[0]}_{item[1]}.mp4", preset='superfast', codec='libx264', audio_codec='aac')
        clip2.write_videofile(f"subclips/end_{item[0]}_{item[1]}.mp4", preset='superfast', codec='libx264', audio_codec='aac')

        
    else:
        
        # Load the audio from another video file
        
        img = 'images/' + interval_words[item[0]]['keywords'] + '.jpeg'

        resize_and_crop(img)
        
#         clip = ImageClip(img).set_fps(30).set_duration(val['end'] - val['start'])
        prev_index_end = interval_words[breaking_points[index-1][1]]['end']
        current_end = interval_words[breaking_points[index+1][0]]['start']
        
        print(prev_index_end,current_end )
        
        audio_clip = VideoFileClip('subclips/cropped.mp4').subclip(prev_index_end, current_end).audio
        
        clip = ImageClip(img).set_fps(30).set_duration(current_end - prev_index_end)
        clip = Zoom(clip,mode='in',position='center',speed=2) #zoom function above
        # Set the audio to the image clip
        clip = clip.set_audio(audio_clip)

        
        clip.write_videofile(f'subclips/{item[0]}_{item[0]}.mp4',preset='superfast', codec='libx264', audio_codec='aac')

        

0
92.41999816894531
102.5999984741211
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Building video subclips/0_1.mp4.
MoviePy - Writing audio in 0_1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/0_1.mp4



Moviepy - Done !
Moviepy - video ready subclips/0_1.mp4
Moviepy - Building video subclips/end_0_1.mp4.
MoviePy - Writing audio in end_0_1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/end_0_1.mp4



Moviepy - Done !
Moviepy - video ready subclips/end_0_1.mp4
1
102.5999984741211 107.94000244140625
Moviepy - Building video subclips/2_2.mp4.
MoviePy - Writing audio in 2_2TEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready subclips/14_14.mp4
6
169.74000549316406
185.05999755859375
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Building video subclips/15_17.mp4.
MoviePy - Writing audio in 15_17TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/15_17.mp4



Moviepy - Done !
Moviepy - video ready subclips/15_17.mp4
Moviepy - Building video subclips/end_15_17.mp4.
MoviePy - Writing audio in end_15_17TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/end_15_17.mp4



Moviepy - Done !
Moviepy - video ready subclips/end_15_17.mp4
time: 22.5 s (started: 2024-05-25 14:08:42 +05:45)


## apply transition

In [283]:
import moviepy.editor as mpe

def apply_transition(prev,nextt, transition, output):

    clip1 = mpe.VideoFileClip(prev)
    clip2 = mpe.VideoFileClip(nextt)
    transition_clip = mpe.VideoFileClip(transition)

    # Apply mask to remove the green background from the transition clip
    masked_transition = transition_clip.fx(mpe.vfx.mask_color, color=[0, 0, 0], thr=100, s=5)

    # Define the duration of the transition
    transition_duration = masked_transition.duration

    # Ensure the transition does not exceed the duration of the clips
    if transition_duration > clip1.duration or transition_duration > clip2.duration:
        raise ValueError("Transition duration exceeds the duration of one or both video clips.")

    # Calculate the start times for the transition
    transition_start_clip1 = clip1.duration - transition_duration + 0.8
    transition_start_clip2 = transition_duration

    # Set the start time for the masked transition clip
    masked_transition = masked_transition.set_start(transition_start_clip1)

    # Create the composite video with the transition
    composite_transition = mpe.CompositeVideoClip([
        clip1,
        masked_transition,
        clip2.set_start(clip1.duration)
#         clip2
    ]).set_duration(clip1.duration + clip2.duration)


    composite_transition.write_videofile(output, codec='libx264', 
                         audio_codec='aac', preset='superfast')

time: 2.44 ms (started: 2024-05-25 13:05:57 +05:45)


In [284]:
breaking_points

[(0, 1), (2, 2), (3, 5), (6, 6), (7, 13), (14, 14), (15, 17)]

time: 2.88 ms (started: 2024-05-25 13:05:58 +05:45)


In [285]:
transition_video = 'clips/lightleaks/checklight1.mp4'

for index,item in enumerate(breaking_points):
    first = item[0]
    second = item[1]
    
    prev = index-1
    
    if first == second:
        prev_video = f'subclips/end_{breaking_points[prev][0]}_{breaking_points[prev][1]}.mp4'
        nextt_video = f'subclips/{first}_{second}.mp4'
        output = f'subclips/{first}_{second}_new.mp4'

        print('Combinig ',prev_video, nextt_video)
        apply_transition(prev_video,nextt_video, transition_video, output)
    
    

Combinig  subclips/end_0_1.mp4 subclips/2_2.mp4
Moviepy - Building video subclips/2_2_new.mp4.
MoviePy - Writing audio in 2_2_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/2_2_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/2_2_new.mp4
Combinig  subclips/end_3_5.mp4 subclips/6_6.mp4
Moviepy - Building video subclips/6_6_new.mp4.
MoviePy - Writing audio in 6_6_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/6_6_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/6_6_new.mp4
Combinig  subclips/end_7_13.mp4 subclips/14_14.mp4
Moviepy - Building video subclips/14_14_new.mp4.
MoviePy - Writing audio in 14_14_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/14_14_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/14_14_new.mp4
time: 29.7 s (started: 2024-05-25 13:05:58 +05:45)


In [286]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

# List of video file paths to be concatenated
video_files = []

for item in breaking_points:
    first = item[0]
    second = item[1]
    
    
    if first == second:
        video_files.append(f'subclips/{first}_{second}_new.mp4')
    else:
        video_files.append(f'subclips/{first}_{second}.mp4')



# Load each video file into a VideoFileClip object
clips = [VideoFileClip(video) for video in video_files]

# Concatenate the list of video clips into a single video
final_clip = concatenate_videoclips(clips, method="compose")

# Write the final video file
final_clip.write_videofile("subclips/final_video_new.mp4", preset='superfast', codec='libx264', audio_codec='aac' )


Moviepy - Building video subclips/final_video_new.mp4.
MoviePy - Writing audio in final_video_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/final_video_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/final_video_new.mp4
time: 20.8 s (started: 2024-05-25 13:06:58 +05:45)


## add transcript

### whispeer the audio

In [181]:
from openai import OpenAI
client = OpenAI()

audio_file= open("subclips/final_video_new.mp4", "rb")
transcript = client.audio.transcriptions.create(
  file=audio_file,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["word"]
)

In [182]:

import pickle
with open('shorttranscript.pkl', 'wb') as file:
    pickle.dump(transcript, file)

In [183]:
transcript.words

[{'word': 'a', 'start': 0.0, 'end': 0.5},
 {'word': 'thinker', 'start': 0.5, 'end': 1.0199999809265137},
 {'word': 'et', 'start': 1.3200000524520874, 'end': 1.3600000143051147},
 {'word': 'cetera', 'start': 1.3600000143051147, 'end': 1.3600000143051147},
 {'word': 'So', 'start': 2.299999952316284, 'end': 2.4800000190734863},
 {'word': 'I', 'start': 2.4800000190734863, 'end': 2.819999933242798},
 {'word': 'like', 'start': 2.819999933242798, 'end': 3.119999885559082},
 {'word': 'the', 'start': 3.119999885559082, 'end': 3.5999999046325684},
 {'word': 'model', 'start': 3.5999999046325684, 'end': 3.940000057220459},
 {'word': 'of', 'start': 3.940000057220459, 'end': 4.360000133514404},
 {'word': 'life', 'start': 4.360000133514404, 'end': 4.539999961853027},
 {'word': 'that', 'start': 4.539999961853027, 'end': 4.800000190734863},
 {'word': 'the', 'start': 4.800000190734863, 'end': 5.179999828338623},
 {'word': 'ancients', 'start': 5.179999828338623, 'end': 5.400000095367432},
 {'word': 'had'

In [184]:
combined_transcript = []
new_transcript = transcript.words


for index, item in enumerate(new_transcript):
    start = item['start']
    end = item['end']
    duration = end - start
    
#     print(item['word'], duration)

    if index == 0: 
        continue
    if duration == 0:
        new_transcript[index-1]['end'] = new_transcript[index]['end']
        new_transcript[index-1]['word'] += " " + new_transcript[index]['word']
        new_transcript[index]['del'] = True
        
        
    

In [185]:
new_transcript

[{'word': 'a', 'start': 0.0, 'end': 0.5},
 {'word': 'thinker', 'start': 0.5, 'end': 1.0199999809265137},
 {'word': 'et cetera', 'start': 1.3200000524520874, 'end': 1.3600000143051147},
 {'word': 'cetera',
  'start': 1.3600000143051147,
  'end': 1.3600000143051147,
  'del': True},
 {'word': 'So', 'start': 2.299999952316284, 'end': 2.4800000190734863},
 {'word': 'I', 'start': 2.4800000190734863, 'end': 2.819999933242798},
 {'word': 'like', 'start': 2.819999933242798, 'end': 3.119999885559082},
 {'word': 'the', 'start': 3.119999885559082, 'end': 3.5999999046325684},
 {'word': 'model', 'start': 3.5999999046325684, 'end': 3.940000057220459},
 {'word': 'of', 'start': 3.940000057220459, 'end': 4.360000133514404},
 {'word': 'life', 'start': 4.360000133514404, 'end': 4.539999961853027},
 {'word': 'that', 'start': 4.539999961853027, 'end': 4.800000190734863},
 {'word': 'the', 'start': 4.800000190734863, 'end': 5.179999828338623},
 {'word': 'ancients', 'start': 5.179999828338623, 'end': 5.4000000

In [186]:
## delete the repeated one

combined_transcript = [item for item in new_transcript if not item.get('del', False) ]

## add subtitle

In [187]:
import sys

from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, AudioFileClip, CompositeAudioClip



def create_subtitle_clips(subtitles, videosize,fontsize=50, font='Helvetica-BoldOblique', color='white', debug = False):
    subtitle_clips = []

    for subtitle in subtitles:
        start_time = subtitle['start']
        end_time = subtitle['end']
        duration = end_time - start_time

        video_width, video_height = videosize
        
        text_clip = TextClip(subtitle['word'], fontsize=fontsize, font=font, color=color ,method='caption').set_start(start_time).set_duration(duration)
        subtitle_x_position = 'center'
        subtitle_y_position = video_height* 4 / 5 

        text_position = (subtitle_x_position, subtitle_y_position)                    
        subtitle_clips.append(text_clip.set_position(text_position))

    return subtitle_clips

In [188]:
# Load video and SRT file
video = VideoFileClip('subclips/final_video_new.mp4')
subtitles = combined_transcript

# begin,end= mp4filename.split(".mp4")
output_video_file = "subclips/subtitled_video_final.mp4"

print ("Output file name: ",output_video_file)

# Create subtitle clips
subtitle_clips = create_subtitle_clips(subtitles,video.size)

# Add subtitles to the video
final_video = CompositeVideoClip([video] + subtitle_clips)


# Load background music
background_music = AudioFileClip('')

# Write output video file
final_video.write_videofile(output_video_file,  preset='superfast', codec='libx264', audio_codec='aac')

Output file name:  subclips/subtitled_video_final.mp4


OSError: MoviePy error: the file  could not be found!
Please check that you entered the correct path.

In [73]:

background_music = AudioFileClip('subclips/bg_music.mp4')

In [200]:

# Load video and SRT file
video = VideoFileClip('subclips/final_video_new.mp4')

audio = AudioFileClip("Video.mp4")

start_time = new_result[0]['transcript'][0]['start']
end_time = new_result[0]['transcript'][-1]['end']

audio_segment = audio.subclip(start_time, end_time)

subtitles = combined_transcript

output_video_file = "subclips/subtitled_video_final.mp4"
print("Output file name:", output_video_file)

# Create subtitle clips
subtitle_clips = create_subtitle_clips(subtitles, video.size)

# Add subtitles to the video
final_video = CompositeVideoClip([video] + subtitle_clips)

# Load background music
background_music = AudioFileClip('subclips/bg_music.mp4')

# Reduce the volume of the background music
background_music = background_music.volumex(0.2)  # Adjust volume as needed

# Combine original audio with background music
# We make the background music match the duration of the video
bg_music = background_music.set_duration(final_video.duration)
combined_audio = CompositeAudioClip([audio_segment, bg_music])

# Set the combined audio as the audio for the final video
final_video = final_video.set_audio(combined_audio)

# Write output video file
final_video.write_videofile(output_video_file, preset='superfast', codec='libx264', audio_codec='aac')


Output file name: subclips/subtitled_video_final.mp4
Moviepy - Building video subclips/subtitled_video_final.mp4.
MoviePy - Writing audio in subtitled_video_finalTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/subtitled_video_final.mp4



Moviepy - Done !
Moviepy - video ready subclips/subtitled_video_final.mp4


## testing cut and join

In [ ]:
128.9199981689453 164.66000366210938 gg (7, 13)
164.66000366210938 169.74000549316406

In [280]:
ffmpeg_extract_subclip("subclips/cropped.mp4", 160, 164.66000, targetname=f"subclips/test/1.mp4",codec='libx264', audio_codec='aac' )
ffmpeg_extract_subclip("subclips/cropped.mp4", 164.66000, 169.74000549316406, targetname=f"subclips/test/2.mp4", codec='libx264', audio_codec='aac')

TypeError: ffmpeg_extract_subclip() got an unexpected keyword argument 'codec'

time: 40.3 ms (started: 2024-05-25 12:50:01 +05:45)


In [275]:
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import concatenate_videoclips

# Cut the video into two parts using MoviePy
video = VideoFileClip("subclips/cropped.mp4")

# Extract the subclips
clip1 = video.subclip(160, 164.66000366210938)
clip2 = video.subclip(164.66000366210938, 169.74)

# Write the subclips to files (optional, for verification)
clip1.write_videofile("subclips/test/1.mp4", preset='superfast', codec='libx264', audio_codec='aac')
clip2.write_videofile("subclips/test/2.mp4", preset='superfast', codec='libx264', audio_codec='aac')

Moviepy - Building video subclips/test/1.mp4.
MoviePy - Writing audio in 1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/test/1.mp4



Moviepy - Done !
Moviepy - video ready subclips/test/1.mp4
Moviepy - Building video subclips/test/2.mp4.
MoviePy - Writing audio in 2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/test/2.mp4



Moviepy - Done !
Moviepy - video ready subclips/test/2.mp4
time: 2.75 s (started: 2024-05-25 12:45:12 +05:45)


In [ ]:

# Alternatively, skip writing subclips to files and concatenate directly



# Concatenate the subclips
final_clip = concatenate_videoclips([clip1, clip2], method="compose")

# Write the final combined video
final_clip.write_videofile("subclips/test/gg.mp4", preset='superfast', codec='libx264', audio_codec='aac')

In [278]:
## join


# Load each video file into a VideoFileClip object
clips = [VideoFileClip("subclips/test/1.mp4"), VideoFileClip("subclips/test/2.mp4")]


# Concatenate the list of video clips into a single video
final_clip = concatenate_videoclips(clips, method="compose")

# Write the final video file
final_clip.write_videofile("subclips/test/gg1.mp4", preset='superfast', codec='libx264', audio_codec='aac' )


Moviepy - Building video subclips/test/gg1.mp4.
MoviePy - Writing audio in gg1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/test/gg1.mp4



Moviepy - Done !
Moviepy - video ready subclips/test/gg1.mp4
time: 2.87 s (started: 2024-05-25 12:46:50 +05:45)


In [197]:
start = new_result[0]['transcript'][0]['start']


92.41999816894531

In [198]:
end = new_result[0]['transcript'][-1]['end']

In [199]:
end

182.5800018310547

In [279]:
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import concatenate_videoclips

# Cut the video into two parts using MoviePy
video = VideoFileClip("subclips/cropped.mp4")

# Extract the subclips
clip1 = video.subclip(160, 164.66)
clip2 = video.subclip(164.66, 169.74)

# Write the subclips to files (optional, for verification)
clip1.write_videofile("subclips/test/1.mp4", preset='superfast', codec='libx264', audio_codec='aac')
clip2.write_videofile("subclips/test/2.mp4", preset='superfast', codec='libx264', audio_codec='aac')

# Alternatively, skip writing subclips to files and concatenate directly

# Concatenate the subclips
final_clip = concatenate_videoclips([clip1, clip2], method="compose")

# Write the final combined video
final_clip.write_videofile("subclips/test/gg2.mp4", preset='superfast', codec='libx264', audio_codec='aac')


Moviepy - Building video subclips/test/1.mp4.
MoviePy - Writing audio in 1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/test/1.mp4



Moviepy - Done !
Moviepy - video ready subclips/test/1.mp4
Moviepy - Building video subclips/test/2.mp4.
MoviePy - Writing audio in 2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/test/2.mp4



Moviepy - Done !
Moviepy - video ready subclips/test/2.mp4
Moviepy - Building video subclips/test/gg2.mp4.
MoviePy - Writing audio in gg2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/test/gg2.mp4



Moviepy - Done !
Moviepy - video ready subclips/test/gg2.mp4
time: 5.22 s (started: 2024-05-25 12:48:38 +05:45)
